# Non-Ergodic GMM Regression (Type-1) using STAN, PyStan Library

This notebook is used to estimate the non-ergodic coefficients and hyper-parameters of a type-1 non-ergodic GMM though Gaussian Process regression using the Bayesian software STAN. 

The Type-1 GMM compnents are:
- The non-ergodic effects are modeled by:
  - $\delta c_{0}$:    constant shift of non-ergodic GMM with respect to base ergodic GMM
  - $\delta c_{1,E}$:  spatially varying eathquake adjustment as a function of the earthquake coordinates
  - $\delta c_{1a,S}$: spatially varying site adjustment as a function of the site coordinates
  - $\delta c_{1b,S}$: spatially independent site adjustment as a function of the station id
- The aleatory variability is captured by:
  - $\delta B^0_{e}$: between-event aleatory term
  - $\delta W^0_{e,s}$: within-event aleatory term
- The non-ergodic hyperparameters are:
  - $\ell_{1,E}$:    correlation lenght of spatially varying earthquake adjustment
  - $\ell_{1a,S}$:   correlation lenght of spatially varying site adjustment
  - $\omega_{1,E}$:  scale of spatially varying earthquake adjustment 
  - $\omega_{1a,S}$: scale of spatially varying site adjustment
  - $\omega_{1b,S}$: scale of spatially independent site adjustment
  - $\tau_{0}$:      standard deviation of between-event aleatory
  - $\phi_{0}$:      standard deviation of within-event aleatory

**Disclaimer:** The non-ergodic regression is perfomed on the total residuals (column ``res_name``) of a base ergodic GMM. 
The total regression residuals ($\epsilon_{tot}$) are defined as:
$$
\epsilon_{tot} = log(IM) - \mu_{erg}(M,R_{rup},V_{S30}, ...) 
$$
where $IM$ is the intesity parameter of interest (e.g. PGA, PSA) and $\mu_{erg}$ is mean functional form of the ergodic GMM in log space. 


This script was developed as part of the Non-ergodic Methodology and Modeling Tools research project summarized in the report by Lavrentiadis G., Kuehn N., Bozorgnia Y., Seylabi E., Meng X., Goulet C., and Kottke A. (2022), "Non‐ergodic Methodology and Modeling Tools (Report GIRS-2022-04)." Natural Hazards Risk and Resiliency Research Center, University of California, Los Angeles.

Support provided by the California Department of Transportation (Caltrans) and Pacific Gas and Electric Company (PG&E) is gratefully acknowledged. 

## Load required libraries

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
#user functions
sys.path.insert(0,'../Python_lib/regression/pystan')
sys.path.insert(0,'../Python_lib/plotting')
import pylib_contour_plots as pycplt

## User Input, Definition of Regression Parameters 
The 1st synthetic dataset based the NGAWest2 CA metadata for the type-1 NGMM is used as an example. The user should update the variables in this section to point to the regression dataset of interest.

In [3]:
#regression name
reg_name  = 'example_regression_ds1_pystan'

#regression dataset
flatfile_fname = '../../Data/Flatfiles/examp_datasets/CatalogNGAWest3CALite_synthetic_data_ngmm1_small_corr_len_Y1.csv'
flatfile_fname = '../../Data/Atkinson_resid_trim_inside.csv'

#output directory
dir_out = '../../Data/Regression/example_ds1_pystan/'

#filename for stan regression code
sm_fname = '../Stan_lib/regression_stan_model1_unbounded_hyp_chol_efficient.stan'
from regression_pystan_model1_unbounded_hyp import RunStan

#stan parameters
runstan_flag  = True
pystan_ver    = 2
res_name      = 'tot_1'
n_iter        = 100
n_chains      = 4
adapt_delta   = 0.8
max_treedepth = 8
#parallel options
flag_parallel = True
# flag_parallel = False


## Load Files

In [4]:
#load flatfile
df_flatfile = pd.read_csv(flatfile_fname)
print(len(df_flatfile))
df_flatfile["staLoc"] = df_flatfile['staLat'].astype(str) + ',' + df_flatfile['staLon'].astype(str) 
df_flatfile = df_flatfile[df_flatfile["ssn"].isin(df_flatfile[['ssn', 'staLoc']].drop_duplicates('staLoc')['ssn'])]
df_flatfile = df_flatfile.drop(columns='staLoc')
len(df_flatfile)

26908


26764

## Preprocessing 
This section can be used by the user for any pre-porcessing steps of the regression dataset.

In [5]:
#keep only North records of NGAWest2
# df_flatfile = df_flatfile.loc[df_flatfile.dsid==0,:]



## Run Regression

In [6]:
print(f'Running regression for:%s'%(reg_name))

#run time start
run_t_strt = time.time()
    
#run stan model
RunStan(df_flatfile, sm_fname, reg_name, dir_out, res_name, 
        runstan_flag=runstan_flag, n_iter=n_iter, n_chains=n_chains,
        adapt_delta=adapt_delta, max_treedepth=max_treedepth,
        pystan_ver=pystan_ver, pystan_parallel=flag_parallel)
       
#run time end
run_t_end = time.time()

#compute run time
run_tm = (run_t_end - run_t_strt)/60
  
#log run time
df_run_info = pd.DataFrame({'computer_name':os.uname()[1],'reg_name':reg_name,'run_time':run_tm}, 
                            index=[1])
                           
#write out run info
fname_reginfo   = '%s/run_info.csv'%(dir_out)
df_run_info.reset_index(drop=True).to_csv(fname_reginfo, index=False)

print(f'Completed regression for: %s'%(reg_name))

Running regression for:example_regression_ds1_pystan


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_52782a0a8916039e5a19073b767372d2 NOW.
In file included from /home/cking/.cache/pypoetry/virtualenvs/ngmm-tools-USM35OYO-py3.10/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /home/cking/.cache/pypoetry/virtualenvs/ngmm-tools-USM35OYO-py3.10/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/cking/.cache/pypoetry/virtualenvs/ngmm-tools-USM35OYO-py3.10/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /tmp/tmp040epfe6/stanfit4anon_model_52782a0a8916039e5a19073b767372d2_7480325089555034861.cpp:1287:
/home/cking/.cache/pypoetry/virtualenvs/ngmm-tools-USM35OYO-py3.10/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Usi


Gradient evaluation took 6.10182 seconds
1000 transitions using 10 leapfrog steps per transition would take 61018.2 seconds.
Adjust your expectations accordingly!


         three stages of adaptation as currently configured.
         Reducing each adaptation stage to 15%/75%/10% of
         the given number of warmup iterations:
           init_buffer = 7
           adapt_window = 38
           term_buffer = 5


Gradient evaluation took 6.05931 seconds
1000 transitions using 10 leapfrog steps per transition would take 60593.1 seconds.
Adjust your expectations accordingly!


         three stages of adaptation as currently configured.
         Reducing each adaptation stage to 15%/75%/10% of
         the given number of warmup iterations:
           init_buffer = 7
           adapt_window = 38
           term_buffer = 5


Gradient evaluation took 7.05948 seconds
1000 transitions using 10 leapfrog steps per transition would take 70594.8 seconds.
Adjust your expectations accordingly!




Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'unknown file name' at line 154)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Iteration:  3 / 100 [  3%]  (Warmup)
Iteration:  1 / 100 [  1%]  (Warmup)


...

Iteration:  3 / 100 [  3%]  (Warmup)


Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'unknown file name' at line 135)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Iteration:  4 / 100 [  4%]  (Warmup)
Iteration:  5 / 100 [  5%]  (Warmup)
Iteration:  6 / 100 [  6%]  (Warmup)
Iteration:  7 / 100 [  7%]  (Warmup)
Iteration:  7 / 100 [  7%]  (Warmup)
Iteration:  8 / 100 [  8%]  (Warmup)
Iteration:  9 / 100 [  9%]  (Warmup)
Iteration:  6 / 100 [  6%]  (Warmup)
Iteration:  8 / 100 [  8%]  (Warmup)
Iteration:  9 / 100 [  9%]  (Warmup)
Iteration: 10 / 100 [ 10%]  (Warmup)
Iteration: 11 / 100 [ 11%]  (Warmup)
Iteration: 10 / 100 [ 10%]  (Warmup)
Iteration: 11 / 100 [ 11%]  (Warmup)
Iteration:  7 / 100 [  7%]  (Warmup)
Iteration:  7 / 100 [  7%]  (Warmup)
Iteration:  8 / 100 [  8%]  (Warmup)
Iteration: 12 / 100 [ 12%]  (Warmup)
Iteration: 12 / 100 [ 12%]  (Warmup)
Iteration:  8 / 100 [  8%]  (Warmup)
Iteration:  9 / 100 [  9%]  (Warmup)
Iteration: 13 / 100 [ 13%]  (Warmup)
Iteration: 13 / 100 [ 13%]  (Warmup)
Iteration:  9 / 100 [  9%]  (Warmup)
Iteration: 10 / 100 [ 10%]  (Warmup)
Iteration: 14 / 100 [ 14%]  (Warmup)
Iteration: 14 / 100 [ 14%]  (Warmup)
I

To run all diagnostics call pystan.check_hmc_diagnostics(fit)
/home/cking/sp-effects/ngmm_tools/Analyses/Regression/../Python_lib/regression/pystan/regression_pystan_model1_unbounded_hyp.py:167: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump({'model' : stan_model, 'fit' : stan_fit}, f, protocol=-1)


AttributeError: 'DataFrame' object has no attribute 'append'

In [15]:
import stan

## Regression Review
The next code chunks produce summary plots to review the regression fit

In [ ]:
%matplotlib inline

#load non-ergodic coefficients and residuals
fname_hparam  = dir_out + reg_name + '_stan_hyperparameters.csv'
fname_gmotion = dir_out + reg_name + '_stan_residuals.csv'
fname_coeff   = dir_out + reg_name + '_stan_coefficients.csv'
df_hparam  = pd.read_csv(fname_hparam)
df_gmotion = pd.read_csv(fname_gmotion, index_col=0)
df_coeff   = pd.read_csv(fname_coeff, index_col=0)

#merge gm-flatfile with non-ergodic coeffs and res 
df_gmotion = pd.merge(df_flatfile[['mag','Rrup','Vs30']], df_gmotion, left_index=True, right_index=True)
df_coeff   = pd.merge(df_flatfile[['mag','Rrup','Vs30']], df_coeff,   left_index=True, right_index=True)

#print mean and standard deviations of aleatory terms
print(f'Between-event mean:\t %.3f'%df_gmotion.res_between.mean())
print(f'Within-event mean:\t %.3f'%df_gmotion.res_within.mean())
print(f'Between-event std:\t %.3f'%df_gmotion.res_between.std())
print(f'Within-event std:\t %.3f'%df_gmotion.res_within.std())

### Non-ergodic Residuals

In [ ]:
#between-event residuals 
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(df_gmotion.mag, df_gmotion.res_between)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel('magnitude',               fontsize=30)
ax.set_ylabel('between-event residuals', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

#within-event residuals vs mag
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(df_gmotion.mag, df_gmotion.res_within)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel('magnitude',               fontsize=30)
ax.set_ylabel('within-event residuals', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

#within-event residuals vs dist
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(df_gmotion.Rrup, df_gmotion.res_within)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel('rupture distance',       fontsize=30)
ax.set_ylabel('within-event residuals', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

### Non-ergodic Coefficients

In [ ]:
#spatially varying earthquake adjustment
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(df_coeff.mag, df_coeff.dc_1e_mean)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel('magnitude',              fontsize=30)
ax.set_ylabel(r'$\mu(\delta c_{1,E})$', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

#spatially varying site adjustment
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(np.log(df_coeff.Vs30), df_coeff.dc_1as_mean)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel(r'$\ln(V_{S30}$',          fontsize=30)
ax.set_ylabel(r'$\mu(\delta c_{1a,S})$', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

#spatially varying site adjustment
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(np.log(df_coeff.Vs30), df_coeff.dc_1bs_mean)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel(r'$\ln(V_{S30})$',         fontsize=30)
ax.set_ylabel(r'$\mu(\delta c_{1b,S})$', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

In [ ]:
#earthquake indices
_, eq_idx  = np.unique(df_coeff[['eqid']].values, axis=0, return_index=True)
#station indices
_, sta_idx = np.unique(df_coeff[['ssn']].values, axis=0, return_index=True)

#spatially varying earthquake adjustment
#---------------------
#mean
#---   ---   ---   ---
cbar_label = r'$\mu(\delta c_{1,E})$'
data2plot  = df_coeff[['eqLat','eqLon','dc_1e_mean']].values[eq_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='RdYlBu_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()

#epistemic uncertainty
#---   ---   ---   ---
cbar_label = r'$\psi(\delta c_{1,E})$'
data2plot  = df_coeff[['eqLat','eqLon','dc_1e_sig']].values[eq_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='Purples_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()

In [ ]:
#spatially varying site adjustment
#---------------------
#mean
#---   ---   ---   ---
cbar_label = r'$\mu(\delta c_{1a,S})$'
data2plot  = df_coeff[['staLat','staLon','dc_1as_mean']].values[sta_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='RdYlBu_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()

#epistemic uncertainty
#---   ---   ---   ---
cbar_label = r'$\psi(\delta c_{1a,S})$'
data2plot  = df_coeff[['staLat','staLon','dc_1as_sig']].values[sta_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='Purples_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()

In [ ]:
#spatially independent site adjustment
#---------------------
#mean
#---   ---   ---   ---
cbar_label = r'$\mu(\delta c_{1b,S})$'
data2plot  = df_coeff[['staLat','staLon','dc_1bs_mean']].values[sta_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='RdYlBu_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()

#epistemic uncertainty
#---   ---   ---   ---
cbar_label = r'$\psi(\delta c_{1b,S})$'
data2plot  = df_coeff[['staLat','staLon','dc_1bs_mean']].values[sta_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='Purples_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()